## Author: Ritvik Kharkar

In [ ]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint
from random import random
import matplotlib.pyplot as plt
from time import time
import pandas as pd
import pickle
from scipy.stats import pearsonr

In [ ]:
def calculate_lambda_at_t_val(params, lambda_0, F, T_vals, t):
    """
    INPUTS:
        params: ((k+1,)) parameters. First k are alpha then delta
        lambda_0: the baseline lambda
        F: (N x k) features
        T_vals: (N x 1) event times
        t: real number, current time stamp
        lambda_0: the baseline event intensity
    
    OUTPUT:
        value of the lambda function at t
    """
    
    #get alpha and delta
    alpha = params[:-1].reshape(-1,1)
    delta = params[-1]
    
    #create the event times mask
    mask = (T_vals <= t)
    
    #get the T vals which are before the current time
    T_vals_lim = T_vals[mask]
    
    #get the features which are before the current time
    F_lim = F[mask]
    
    #get sum of weighted features
    weights_lim = np.exp(delta * T_vals_lim)
    sum_weighted_features_lim = (weights_lim * F_lim).sum(axis=0).reshape(-1,1)
    
    #get weighted alpha
    weighted_alpha = (alpha * np.exp(-delta * t)).reshape(-1,1)
    
    #compute dot product
    lambda_val = lambda_0 + (sum_weighted_features_lim * weighted_alpha).sum()
        
    #return final lambda value
    return lambda_val

In [ ]:
def calculate_lambda_at_T_vals(params, lambda_0, F, T_vals):
    """
    INPUTS:
        params: ((k+1,)) parameters. First k are alpha then delta
        lambda_0: the baseline lambda
        F: (N x k) features
        T_vals: (N x 1) event times
        lambda_0: real number, baseline event intensity
    
    OUTPUT:
        value of the lambda function at each t in T_vals, (N x 1)
    """
    
    #get alpha and delta
    alpha = params[:-1].reshape(-1,1)
    delta = params[-1]
    
    #get the weighted scores
    weights = np.exp(delta * T_vals)
    weighted_scores = F * weights
    
    #get cumulative sum of weighted scores
    cumul_sum_weighted_scores = np.cumsum(weighted_scores, axis=0)
    
    #get the weighted alpha
    weighted_alpha = np.transpose(alpha) * (1/weights)
    
    #get elementwise product and sum each row
    lambda_vals = lambda_0 + (weighted_alpha * cumul_sum_weighted_scores).sum(axis=1).reshape(-1,1)
    
    #return the lambda_values
    return lambda_vals

In [ ]:
def calculate_neg_log_likelihood(params, lambda_0, T_vals, diagnostics=False):
    """
    INPUTS:
        params: ((k+1,)) parameters. First k are alpha and then delta
        lambda_0: the baseline lambda
        T_vals: ((N+1) x 1) event times
    
    OUTPUT:
        value of the negative log likelihood function
    """
    
    #get alpha and delta and lambda_0
    alpha = params[0]
    delta = params[-1]
    
    #get weights
    weights = np.exp(delta*T_vals)
    
    #get inverse weights
    inv_weights = 1 / weights
    
    #get difference between inverse weights
    diff_inv_weights = inv_weights[:-1] - inv_weights[1:]
    
    #get cumulative sum of weights
    cumul_sum_weights_inc_first = np.cumsum(weights)[:-1].reshape(-1,1)
    cumul_sum_weights_not_inc_first = np.cumsum(weights)[1:].reshape(-1,1)
    
    #calculate the sum term
    sum_term = np.sum(np.log(lambda_0*np.ones_like(diff_inv_weights) + alpha*inv_weights[1:]*cumul_sum_weights_not_inc_first))
    
    #calculate the integral term 
    integral_term = lambda_0*T_vals[-1][0] + (alpha / delta) * np.sum(cumul_sum_weights_inc_first * diff_inv_weights)
    
    if diagnostics:
        print('Sum Term: %s'%sum_term)
        print('Integral Term: %s'%integral_term)
        
        print('inv_weights: \n%s'%inv_weights[1:])
        print('cumul_sum_weights_not_inc_first: \n%s'%cumul_sum_weights_not_inc_first)
        print('elementwise product: \n%s'%(inv_weights[1:]*cumul_sum_weights_not_inc_first))
        print('inside log: \n%s'%(lambda_0 + alpha*inv_weights[1:]*cumul_sum_weights_not_inc_first))
    
    #return total negative log likelihood
    return integral_term - sum_term

In [ ]:
def simulate_hawkes_process(params, lambda_0, T_vals, F, dt, T, viz=True, single_event=False):
    """
    INPUTS:
        params: ((k+1,)) parameters. First k are alpha then delta
        lambda_0: the baseline lambda
        F: (1 x k) first feature
        T_vals: (1 x 1) first event time
        dt: real number, step size during simulation
        T: end time of simulation
        lambda_0: the baseline event intensity
        viz: boolean for whether we want to display a graph or not
        
        
    OUTPUTS (in order):
        T_vals: final times of all events
        F: final features
    """
    
    #create list of lambda vals for viz
    lambda_vals = []
    
    #create list of time vals for viz
    time_vals = []
    
    #create list of event times for viz
    event_times = []
    
    #get the current time as the first passed in event
    curr_time = T_vals[-1][0] + dt
    
    #while we haven't yet reached out final time
    while curr_time < T:
        
        #append this time val to a list for viz
        time_vals.append(curr_time)
        
        #get the probability that we have an event right now
        lambda_val = calculate_lambda_at_t_val(params, lambda_0, F, T_vals, curr_time)
        prob = lambda_val * dt
        
        #append this lambda val to a list for viz
        lambda_vals.append(lambda_val)
        
        #if we meet that probability, then add an event to the data
        if random() < prob:
            event_times.append(curr_time)
            T_vals = np.concatenate([T_vals, [[curr_time]]], axis=0)
            F = np.concatenate([F, [[1]]], axis=0)
            if single_event:
                return event_times[0]
        
        #increment current time
        curr_time += dt
        
    #return right away if no viz
    if not viz:
        return T_vals, F
    
    #otherwise there is viz
    plt.figure(figsize=(10,6))
    plt.plot(time_vals, lambda_vals)
    plt.xlabel('Time', fontsize=16)
    plt.ylabel('lambda (event intensity)', fontsize=16)

    for t in event_times:
        plt.axvline(t, color='k', linestyle='--', alpha=.5)
        
    return T_vals, F

In [ ]:
def generate_random_params():
    alpha = 1 - np.random.random()
    delta = np.random.random()
    return np.array([alpha, delta])

In [ ]:
def get_recovered_params(T_vals, num_starts, true_params, lambda_0, dev_level=0.1):
    """
    This function starts the minimization process from many random starting points and returns the recovered params
    """
    
    #set the constraints on the params
    A_1 = np.array([[1,1], [0,1], [-1,0]])
    constraint_1 = LinearConstraint(A_1, 0, np.inf)
    
    recovered_params = []
    
    for _ in range(num_starts):
        
        #get starting params
        params_init = generate_random_params()

        try:
            #get recoverd params
            result = minimize(calculate_neg_log_likelihood, x0=params_init, args=(lambda_0, T_vals), method='COBYLA', \
                              constraints=[{'type': 'ineq', 'fun': lambda x: -x[0]},
                                           {'type': 'ineq', 'fun': lambda x: x[1]},
                                           {'type': 'ineq', 'fun': lambda x: x[0] + x[1]}])
            
            if result.success == True:
                recovered_params.append(result.x)
        
        except ValueError as v:
            print(v)
                
    return np.array(recovered_params)

In [ ]:
def get_missing_param(true_params, lambda_0, T_vals, channel_name=''):
    
    t1_start = time()

    for idx in range(true_params.shape[0]):
        
        var = 'alpha' if idx == 0 else 'delta'
        
        param_vals = np.arange(0,1,.01) if var=='delta' else np.arange(-1,0,.01)
        
        losses = np.empty_like(param_vals)
        
        min_pair = [-1,99999999]

        for i in range(param_vals.shape[0]):
            p_copy = true_params.copy()
            p_copy[idx] = param_vals[i]
            losses[i] = calculate_neg_log_likelihood(p_copy, lambda_0, T_vals)
            if losses[i] < min_pair[1]:
                min_pair[0] = param_vals[i]
                min_pair[1] = losses[i]
        
        plt.plot(param_vals, losses)
        plt.title('%s\nTrue Value: %s\nLoss Min Param: %s\nLoss: %s'%(var, true_params[idx], round(min_pair[0],2), round(min_pair[1],2)), fontsize=16)
        plt.xlabel(var, fontsize=14)
        plt.ylabel('Loss', fontsize=14)
        plt.tight_layout()
        plt.show()
        plt.savefig('%s_one_const_var=%s_lambda0=%s_alpha=%s_delta=%s.png'%(channel_name, var, lambda_0, true_params[0], true_params[1]))
        plt.clf()
        #print('Best %s: %s'%(var, round(min_pair[0],2)))
        
    t1_end = time()
    #print('One-Param Time: %s'%(t1_end-t1_start))
        
    #whole grid search
    
    t2_start = time()

    alpha_vals = np.arange(-1,0,.01)

    delta_vals = np.arange(0,1,.01)

    min_params = [-1,-1, 99999999]
    
    points = []

    for alpha in alpha_vals:
        for delta in delta_vals:
            if delta + alpha > 0:
                loss = calculate_neg_log_likelihood(np.array([alpha, delta]), lambda_0, T_vals)
                if not np.isnan(loss):
                    points.append((alpha, delta, loss))
                if loss < min_params[-1]:
                    min_params = [alpha, delta, loss]

    print('Whole Grid Result: %s'%min_params)
    
    t2_end = time()
    
    plt.figure(figsize=(8,8))
    losses = [i[2] for i in points]
    min_loss = min(losses)
    max_loss = max(losses)
    #print('Min Loss: %s'%min_loss)
    #print('Max Loss: %s'%max_loss)
    losses = [(i - min_loss) / (max_loss - min_loss) for i in losses]
    colors = [(l,l,l) for l in losses]
    
    plt.scatter([i[0] for i in points], [i[1] for i in points], color=colors, s=10)
    plt.axvline(min_params[0], color='r', linestyle='--', alpha=0.75)
    plt.axhline(min_params[1], color='r', linestyle='--', alpha=0.75)
    
    plt.title('Grid Search Results:\nalpha=%s, delta=%s'%(round(min_params[0],2), round(min_params[1],2)), fontsize=18)
    plt.xlabel('alpha', fontsize=16)
    plt.ylabel('delta', fontsize=16)
    
    plt.tight_layout()
    plt.savefig('%s_grid_search_lambda_0=%s_alpha=%s_delta=%s.png'%(channel_name, lambda_0, true_params[0], true_params[1]))
    
    #print('Grid Search Time: %s'%(t2_end-t2_start))
    
    return min_params

In [ ]:
def get_next_event_times(T_vals, lambda_0, params, num_sims=1000):
    next_event_times = []
    F_init = np.ones_like(T_vals)
    for _ in range(num_sims): 
        next_event_time = simulate_hawkes_process(params, lambda_0, T_vals, F_init, .01, 10000000, viz=False, single_event=True)
        next_event_times.append(next_event_time)
    return next_event_times

In [ ]:
def simulate_future_event(T_vals, limit, lambda_0, params, num_sims, channel_name):
    num_included = len(T_vals[T_vals < limit]) + 1
    next_event_times = get_next_event_times(T_vals[:num_included].reshape(-1,1), lambda_0, params, num_sims)
    
    avg = round(np.mean(next_event_times), 2)
    dev = round(np.std(next_event_times), 2)
    true = round(T_vals[num_included][0],2 )
    
    """
    plt.figure(figsize=(8,4))
    
    plt.hist(next_event_times, color='cornflowerblue', edgecolor='black', linewidth=2, density=True)
    plt.axvline(avg, alpha=0.75, linestyle='--', color='r')
    plt.axvline(true, alpha=0.75, color='b')
    plt.ylabel('Density', fontsize=16)
    plt.xlabel('Predicted Next Event Time', fontsize=16)
    plt.title('%s - After t=%s days\nAvg. Pred=%s days\nDev. Pred=%s days\nTrue Event Time=%s days'%(channel_name, limit, avg, dev, true), fontsize=20)
    
    plt.tight_layout()
    
    plt.savefig('%s_%sdays_prediction.png'%(channel_name, limit))
    """
    return {'pred': avg, 'true': true, 'dev': dev}

In [ ]:
def gibbs_type_search(true_params, lambda_0, T_vals):
    
    #the alpha sub domain
    #alpha_vals = np.arange(-1,0,.01)
    alpha_vals = np.arange(0,.5,.005)

    #the delta sub domain
    delta_vals = np.arange(0,.5,.005)
    
    #get ogrid to enforce delta > -alpha
    a,d = np.ogrid[0:len(alpha_vals), 0:len(delta_vals)]
    
    #precompute negative log likelihood values
    NLL_vals = np.ones((len(alpha_vals), len(delta_vals)))*10000000000
    
    #print('start compute NLL')
    for i,alpha in enumerate(alpha_vals):
        for j,delta in enumerate(delta_vals):
            if alpha < delta:
                loss = calculate_neg_log_likelihood(np.array([alpha, delta]), lambda_0, T_vals)
                if not np.isnan(loss):
                    NLL_vals[i,j] = loss
                    if loss < 0:
                        print(alpha, delta, loss)
                        print('-----')
    #print('finish compute NLL')
    
    #initial (alpha_idx, delta_idx)
    curr_params = [1,0]
    while delta_vals[curr_params[1]] < alpha_vals[curr_params[0]]:
        curr_params = [np.random.choice(range(len(alpha_vals))), np.random.choice(range(len(delta_vals)))]
        
    #print('Starting at %s'%[alpha_vals[curr_params[0]], delta_vals[curr_params[1]]])
    curr_loss = NLL_vals[curr_params[0], curr_params[1]]
    #print('Loss: %s'%curr_loss)
    #print('=========')

    loss_vals = [curr_loss]
    param_vals = [curr_params]
    
    #set tolerance
    tol = 1.01
    
    #iterate
    max_iters = 100
    curr_iter = 0
    
    start = time()
    
    while curr_iter < max_iters:
        
        ##### CHOOSE NEXT DELTA #####
        
        alpha_index = curr_params[0]
        
        #get valid NLL_vals given this alpha
        valid_indices = delta_vals > -alpha_vals[alpha_index]
        valid_deltas = delta_vals[valid_indices]
        valid_NLL_vals = NLL_vals[alpha_index][valid_indices]
        valid_index_nums = np.nonzero(valid_indices)[0]
        
        #get indices of 10 lowest losses
        mask = valid_NLL_vals < loss_vals[-1] * tol
        
        if mask.sum() <= 1:
            break
        
        valid_NLL_vals = valid_NLL_vals[mask]
        valid_index_nums = valid_index_nums[mask]
        sorted_NLL_inds = valid_NLL_vals.argsort()
        valid_NLL_vals = valid_NLL_vals[sorted_NLL_inds]
        valid_index_nums = valid_index_nums[sorted_NLL_inds]
        
        #get valid weights and normalize
        valid_weights = 1 / valid_NLL_vals
        valid_weights = valid_weights / valid_weights.sum()
        
        chosen_delta_idx = np.random.choice(valid_index_nums, p=valid_weights)
        curr_params[1] = chosen_delta_idx
        #print('update delta: %s'%[alpha_vals[curr_params[0]], delta_vals[curr_params[1]]])
        #print('Loss: %s'%NLL_vals[curr_params[0], curr_params[1]])
        param_vals.append((alpha_vals[curr_params[0]], delta_vals[curr_params[1]]))
        
        ##### CHOOSE NEXT ALPHA #####
        
        delta_index = curr_params[1]
        
        #get valid NLL_vals given this delta
        valid_indices = delta_vals[delta_index] > -alpha_vals
        valid_alphas = alpha_vals[valid_indices]
        valid_NLL_vals = NLL_vals[:,delta_index].flatten()[valid_indices]
        valid_index_nums = np.nonzero(valid_indices)[0]
        
        #get indices of 10 lowest losses
        mask = valid_NLL_vals < loss_vals[-1] * tol
        
        if mask.sum() <= 1:
            break
        
        valid_NLL_vals = valid_NLL_vals[mask]
        valid_index_nums = valid_index_nums[mask]
        sorted_NLL_inds = valid_NLL_vals.argsort()[:10]
        valid_NLL_vals = valid_NLL_vals[sorted_NLL_inds]
        valid_index_nums = valid_index_nums[sorted_NLL_inds]
        
        #get valid weights and normalize
        valid_weights = 1 / valid_NLL_vals
        valid_weights = valid_weights / valid_weights.sum()
        
        chosen_alpha_idx = np.random.choice(valid_index_nums, p=valid_weights)
        curr_params[0] = chosen_alpha_idx
        #print('update alpha: %s'%[alpha_vals[curr_params[0]], delta_vals[curr_params[1]]])
        #print('Loss: %s'%NLL_vals[curr_params[0], curr_params[1]])
        
        loss_vals.append(NLL_vals[curr_params[0], curr_params[1]])
        param_vals.append((alpha_vals[curr_params[0]], delta_vals[curr_params[1]]))
        
        #print('=========')
        
        curr_iter += 1
        
    end = time()
    #print('Total Time: %s'%((end-start)*1000))
    
    min_param_loc = np.where(NLL_vals == NLL_vals.min())
    grid_min_alpha_idx = min_param_loc[0][0]
    grid_min_delta_idx = min_param_loc[1][0]
    grid_min_alpha = alpha_vals[grid_min_alpha_idx]
    grid_min_delta = delta_vals[grid_min_delta_idx]
    
    return param_vals[-1]
    
    plt.figure(figsize=(10,6))
    plt.plot(loss_vals[1:])
    plt.xlabel('Step', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    #plt.title('Gibbs-Type Search\nTrue Params: lambda_0=%s, alpha=%s, delta=%s'%(lambda_0, true_params[0], true_params[1]), fontsize=18)
    #print('Grid Min Loss:', NLL_vals.min())
    #print('Gibbs Min Loss:', loss_vals[-1])
    
    #print('Grid Solution:', (grid_min_alpha, grid_min_delta))
    #print('Gibbs Solution:', (param_vals[-1][0], param_vals[-1][1]))
    
    plt.tight_layout()
    #plt.savefig('Gibbs_Type Search_True Params_lambda_0=%s_alpha=%s_delta=%s.png'%(lambda_0, true_params[0], true_params[1]))
    
    plt.show()
    plt.clf()
    
    
    
    plt.figure(figsize=(5,5))
    
    min_alpha = min([item[0] for item in param_vals]) - .1
    max_alpha = max([item[0] for item in param_vals]) + .1
    
    min_delta = min([item[1] for item in param_vals]) - .1
    max_delta = max([item[1] for item in param_vals]) + .1
    
    for i in range(len(param_vals)-1):
        alpha_last, delta_last = param_vals[i]
        alpha_next, delta_next = param_vals[i+1]
        plt.plot((alpha_last, alpha_next), (delta_last, delta_next), marker = 'o', color='k')
        
    search_start_loc = plt.plot((param_vals[0][0], param_vals[0][0]), (param_vals[0][1], param_vals[0][1]), marker = 'o', color='r', linewidth=.5)
    true_loc = plt.plot((true_params[0], true_params[0]), (true_params[1], true_params[1]), marker = '*', color='orange', markersize=10)
    grid_search_loc = plt.plot((grid_min_alpha, grid_min_alpha), (grid_min_delta, grid_min_delta), marker = 'x', color='magenta', markersize=10)
        
    plt.xlim(min_alpha, max_alpha)
    plt.ylim(min_delta, max_delta)
    plt.xlabel('alpha', fontsize=16)
    plt.ylabel('delta', fontsize=16)
    plt.title('Gibbs Sampler Path\nTrue Params:\nalpha=%s, delta=%s'%(true_params[0], true_params[1]), fontsize=18)
    plt.tight_layout()
    plt.savefig('Gibbs_Sampler_Path_True Params_lambda=%s_alpha=%s_delta=%s.png'%(lambda_0, true_params[0], true_params[1]))
    plt.show()
    

In [ ]:
gibbs_sols = {}
for name, data in channel_name_to_events_list.items():
    print(name)
    trial_sols = []
    for _ in range(10):
        trial_sols.append(gibbs_type_search(None, 0.1, data['series']))
    gibbs_sols[name] = [np.mean([item[i] for item in trial_sols]) for i in range(2)]
    print(gibbs_sols[name])

In [ ]:
channel_name_to_preds = {}
for channel_name in channel_name_to_events_list:
    for time_lim in [20,40]:
        print(channel_name, time_lim)
        result = simulate_future_event(channel_name_to_events_list[channel_name]['series'], time_lim, 0.1, np.array(gibbs_sols[channel_name]), 1000, channel_name)
        channel_name_to_preds[(channel_name, time_lim)] = result
        print('----------')

In [ ]:
channel_names = []
avg_time_between_uploads = []
num_uploads = []
pct_error_20 = []
pct_error_40 = []
estimated_alpha = []
estimated_K = []
areas = []

for channel_name in channel_name_to_events_list:
    channel_names.append(channel_name)
    areas.append(channel_name_to_events_list[channel_name]['area'])
    series = channel_name_to_events_list[channel_name]['series']
    avg_time_between_uploads.append(np.diff(series.flatten()).mean())
    num_uploads.append(len(series))
    sols = gibbs_sols[channel_name]
    estimated_alpha.append(sols[1])
    estimated_K.append(sols[0] / sols[1])
    pct_error_20.append(abs(channel_name_to_preds[(channel_name, 20)]['pred'] - channel_name_to_preds[(channel_name, 20)]['true']) / channel_name_to_preds[(channel_name, 20)]['true'])
    pct_error_40.append(abs(channel_name_to_preds[(channel_name, 40)]['pred'] - channel_name_to_preds[(channel_name, 40)]['true']) / channel_name_to_preds[(channel_name, 40)]['true'])

In [ ]:
df_preds = pd.DataFrame(columns=['channel_name', 'area', 'avg_days_between_uploads', 'num_uploads', 'estimated_alpha', 'estimated_K', 'pct_error_20', 'pct_error_40'], data=zip(channel_names, areas, avg_time_between_uploads, num_uploads, estimated_alpha, estimated_K, pct_error_20, pct_error_40))

In [ ]:
#correlation between percent error 20 and avg days between uploads
pearsonr(df_preds.avg_days_between_uploads, df_preds.pct_error_20)

In [ ]:
#correlation between percent error 40 and avg days between uploads
pearsonr(df_preds.avg_days_between_uploads, df_preds.pct_error_40)

In [ ]:
#correlation between alpha and avg days between uploads
pearsonr(df_preds.estimated_alpha, df_preds.avg_days_between_uploads)

In [ ]:
#correlation between K and avg days between uploads
pearsonr(df_preds.estimated_K, df_preds.avg_days_between_uploads)

In [ ]:
#correlation between pct error 20 and estimated alpha
pearsonr(df_preds.estimated_alpha, df_preds.pct_error_20)

In [ ]:
#correlation between pct error 40 and estimated alpha
pearsonr(df_preds.estimated_alpha, df_preds.pct_error_40)

In [ ]:
#correlation between pct error 20 and estimated K
pearsonr(df_preds.estimated_K, df_preds.pct_error_20)

In [ ]:
#correlation between pct error 40 and estimated K
pearsonr(df_preds.estimated_K, df_preds.pct_error_40)

In [ ]:
df_preds.groupby('area').mean().sort_values('pct_error_20')

In [ ]:
correlation_matrix = np.zeros((len(df_preds.columns[2:]), len(df_preds.columns[2:])))
significant_matrix = np.zeros((len(df_preds.columns[2:]), len(df_preds.columns[2:]))).astype(bool)
for i1,c1 in enumerate(df_preds.columns[2:]):
    for i2,c2 in enumerate(df_preds.columns[2:]):
        res = pearsonr(df_preds[c1], df_preds[c2])
        correlation_matrix[i1,i2] = res[0]
        if res[1] < 0.05:
            significant_matrix[i1,i2] = True

In [ ]:
#Set figure size
plt.figure(figsize=(10,8))
#Specify we would like a heatmap
plt.imshow(correlation_matrix, interpolation = 'nearest', cmap = 'Greys')
#Specify the x and y labels
plt.xticks(range(6), df_preds.columns[2:], rotation = 90, fontsize = 16)
plt.yticks(range(6), df_preds.columns[2:], fontsize = 16)

#include a colorbar for reference
plt.ylim(-.5,5.5)
plt.xlim(-.5,5.5)
plt.colorbar()

for i1 in range(6):
    for i2 in range(6):
        if significant_matrix[i1,i2]:
            plt.scatter([i1],[i2], marker='x', color='r')
            
plt.tight_layout()
plt.savefig('correlation_heatmap.png')

## Simulation

In [ ]:

T_vals_init = np.array([0]).reshape((1,1))
F_init = np.ones((1,1))

lambda_0 = 0.1
alpha = 0.48
K = .4/.48

true_params = np.array([alpha*K, alpha])
T_vals_final, F_final = simulate_hawkes_process(np.array(gibbs_sols[channel_name]), lambda_0, T_vals_init, F_init, .01, 93, viz=True)

gaps = np.diff(T_vals_final.reshape(1,-1))

print('Avg Gap:', np.mean(gaps))
print('Dev Gap:', np.std(gaps))
print('Num Events:', len(T_vals_final))

plt.title('Simulated Hawkes Process\nlambda_0=%s, alpha=%s, K=%s'%(lambda_0, alpha, K), fontsize=20)

plt.tight_layout()

plt.savefig('updated_hawkes_process_sim_lambda_0=%s_alpha=%s_K=%s.png'%(lambda_0, alpha, K))

# Get Real Data

In [ ]:
import pickle

In [ ]:
channel_name_to_events_list = pickle.load( open( "event_times.p", "rb" ) )

In [ ]:
for name, data in channel_name_to_events_list.items():
    channel_name_to_events_list[name]['series'] = np.array(channel_name_to_events_list[name]['series']).reshape((-1,1)) / 24

## Try to recover params - COBYLA

In [ ]:
lambda_0 = 1

num_trials = 1000

start = time()
result = get_recovered_params(allrecipes_T_vals, num_trials, true_params, lambda_0, .5)
end = time()
print('Avg Milliseconds: %s'%str((end - start)/num_trials*1000))

print('Success Rate:', len(result) / num_trials)

print('Avg Recoverd Params:', result.mean(axis=0))

## Grid Search and One-Held Constant

In [ ]:
lambda_0 = 1
true_params = np.array([-0.15, 0.15])

In [ ]:
result = get_missing_param(true_params, lambda_0, allrecipes_T_vals, 'allrecipes')

## Gibbs-Type Search

In [ ]:
loss_vals = gibbs_type_search(None, 0.1, channel_name_to_events_list['Khan Academy']['series'])

## Simulate with Recovered Params

In [ ]:
T_vals_init = np.array([0]).reshape((1,1))
F_init = np.ones((1,1))
recovered_params = np.array([-.02, .05])
T_vals_final, F_final = simulate_hawkes_process(recovered_params, lambda_0, T_vals_init, F_init, .01, allrecipes_T_vals[-1][0], viz=True)

gaps = np.diff(T_vals_final.reshape(1,-1))

avg_gap = round(np.mean(gaps), 2)
dev_gap = round(np.std(gaps), 2)
num_events = len(T_vals_final)

print('Avg Gap:', avg_gap)
print('Dev Gap:', dev_gap)
print('Num Events:', num_events)
plt.title('Hawkes Process from Recovered Params\nYouTube Channel: Allrecipes\nAvg. Gap: %s | Dev. Gap: %s | Num Events: %s'%(avg_gap, dev_gap, num_events), fontsize=20)
plt.tight_layout()